In [16]:
from collections import defaultdict, Counter
import pandas as pd
from nltk import ngrams
import spacy

In [17]:
#!python -m spacy download en
_spacy = spacy.load("en_core_web_sm")

In [ ]:
df = pd.read_csv('adaptive_speak_eval_all.csv')
df.head(2)

In [19]:
refs = df['golden utt'].tolist()
hyps = {
    "h0": df['original utt'].tolist(),
    "h0' 0": df['adapted utt s0'].tolist(),
    # "h0' 1": df['adapted utt s1'].tolist(),
    # "h0' 2": df['adapted utt s2'].tolist()
}

In [68]:
pos = defaultdict(lambda: defaultdict(lambda: Counter()))

for ref in refs:
    pos_seq = [tok.pos_ for tok in _spacy(ref)]
    for N in range(1, 4):
        for ngram in ngrams(pos_seq, N):
            pos[N]['ref'][ngram] += 1

for hyp_type in hyps:
    for hyp in hyps[hyp_type]:
        pos_seq = [tok.pos_ for tok in _spacy(ref)]
        for N in range(1, 4):
            for ngram in ngrams(pos_seq, N):
                pos[N][hyp_type][ngram] += 1

In [35]:
pos_df_1 = pd.DataFrame(pos[1].values(), index=pos[1].keys())
pos_df_2 = pd.DataFrame(pos[2].values(), index=pos[2].keys())
pos_df_3 = pd.DataFrame(pos[3].values(), index=pos[3].keys())

In [ ]:
pos_df_1.plot.barh(figsize=(20,10))